# Introduction to atomman: LAMMPS Potentials

__Lucas M. Hale__, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), _Materials Science and Engineering Division, NIST_.
    
[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

__Table of contents__

1. [Introduction](#section1) 
2. [Loading LAMMPS Potentials](#section2)
3. [Potential Classes](#section3)
4. [User-Defined Potentials](#section4)
5. [potential_LAMMPS Data Model](#section5)

## 1. Introduction <a id='section1'></a>

The atomman package is designed to interact with LAMMPS-compatible interatomic potential implementations and assist with the automatic and dynamic generation of the associated LAMMPS command lines.  Starting with atomman version 1.3.3, the main management of LAMMPS potentials is now handled with the [potentials package](https://github.com/usnistgov/potentials), with atomman inheriting classes for representing LAMMPS potentials and accessing database records from the NIST Interatomic Potentials Repository. These methods allow for pre-defined potentials to be retrieved either from the [remote database potentials.nist.gov](https://potentials.nist.gov/) or from a local directory, and for user-defined potentials to be easily integrated in with the official ones.

**Library Imports**

In [1]:
# Standard libraries
from pathlib import Path
import datetime

# http://www.numpy.org/
import numpy as np             

# https://github.com/usnistgov/atomman
import atomman as am            
import atomman.lammps as lmp
import atomman.unitconvert as uc

# Show atomman version
print('atomman version =', am.__version__)

# Show date of Notebook execution
print('Notebook executed on', datetime.date.today())

atomman version = 1.4.11
Notebook executed on 2024-04-29


## 2. Loading LAMMPS Potentials <a id='section2'></a>


### 2.1. Using load_lammps_potential()

*Added version 1.3.3*

*Updated version 1.4.0* for potentials package version 0.3.0

The function atomman.load_lammps_potential() allows for a LAMMPS potential to be loaded from the potentials database. Depending on the settings used, this function will search for matching potentials from the local database copy and/or potentials.nist.gov.

Search parameters
    
- __name__ (*str or list*) The record name(s) to parse by.  For potential records, the names should correspond to the id with a prefix of "potentials." added to it.
- __key__ (*str or list*) The unique UUID4 record key(s) to parse by. 
- __id__ (*str or list*) The unique record id(s) labeling the records to parse by.
- __potid__ (*str or list*) The unique UUID4 record key(s) for the associated potential records to parse by.
- __potkey__ (*str or list*) The unique record id(s) labeling the associated potential records to parse by.
- __units__ (*str or list*) LAMMPS units option(s) to parse by.
- __atom_style__ (*str or list*) LAMMPS pair_style(s) to parse by.
- __pair_style__ (*str or list*) LAMMPS pair_style(s) to parse by.
- __status__ (*None, str or list*) Limits the search by the status of the LAMMPS implementations: "active", "superseded" and/or "retracted".  By default, only active implementations are returned.  Giving a value of None will return implementations of all statuses.
- __symbols__ (*str or list*) Model symbol(s) to parse by.  Typically correspond to elements for atomic potential models.
- __elements__ (*str or list*) Element(s) in the model to parse by.

Database settings

- __database__ (*atomman.library.Database, optional*) Allows for a previously defined Database object to be used to find the potential.  If not given, a new Database object will be used with the default local and remote interaction settings.
- __local__ (*bool, optional*) Indicates if the local location is to be searched.  Default value matches the value set when the database was initialized.
- __remote__ (*bool, optional*) Indicates if the remote location is to be searched.  Default value matches the value set when the database was initialized.
- __kim_models__ (*list*) A list of full KIM model ids to build LAMMPS potentials for.
- __kim_api_directory__ (*str*) The path to the directory containing a kim-api-collections-management executable to use to identify which KIM models are installed.
- __kim_models_file__ (*str*) The path to a file containing a list of full KIM model ids to build LAMMPS potentials for.
        
Other options

- __pot_dir_stylee__ (*str, optional*) Specifies how the pot_dir values will be set for the retrieved LAMMPS potentials.  Allowed values are 'working', 'id', and 'local'. 
    - 'working' will set all pot_dir = '', meaning parameter files are expected in the working directory when the potential is accessed. 
    - 'id' sets the pot_dir values to match the potential's id. 
    - 'local' sets the pot_dir values to the corresponding local database paths where the files are expected to be found.  Default value is controlled by settings.
- __getfilese__ (*bool, optional*) If True, then the parameter files for the matching potentials will also be copied/downloaded to the potential directory.
- __prompte__ (*bool*) If prompt=True (default) then a screen input will ask for a selection if multiple matching potentials are found.  If prompt=False, then an error will be thrown if multiple matches are found.
- __verbosee__ (*bool, optional*) If True, info messages will be printed during operations.  Default value is False.

Returns

- (*potentials.PotentialLAMMPS*) The potential object to use.


    
    



#### 2.1.1. By id

Each LAMMPS potential in the NIST Interatomic Potentials Repository has a unique id that indicates the associated publication, what the model represents, and which archived version it is for. These id's can be easily found by either searching for potentials using

- [Interatomic Potentials Repository](https://www.ctcms.nist.gov/potentials) and finding the ids for the LAMMPS versions you want to use - the id's in parenthesis that contain "LAMMPS" within them.
- [potentials.nist.gov](https://potentials.nist.gov/explore/keyword/) and in the "Filter by Template" section on the left clicking the box next to potential_LAMMPS and/or potential_LAMMPS_KIM.
- [atomman.library.Database](1.5. Settings and databases.ipynb) (and section 2.2 below) provides additional methods for searching and exploring the hosted potentials from within Python.

In [2]:
potential = am.load_lammps_potential(id='1989--Adams-J-B--Ni--LAMMPS--ipr1')
print(potential)

potential_LAMMPS record named 1989--Adams-J-B--Ni--LAMMPS--ipr1


Note that by default the function will only return currently active versions of LAMMPS potentials, so older versions that have been marked as "superseded" or "retracted" won't show.  Setting the status parameter to None will allow for the old versions to be retrieved. 

In [3]:
# Old LAMMPS version won't be found because it is not "active"
try:
    potential = am.load_lammps_potential(id='2004--Zhou-X-W--Ni--LAMMPS--ipr1')
except Exception as ex:
    print(f'{type(ex).__name__}: {ex.args[0]}')
else:
    print(potential)

ValueError: No matching LAMMPS potentials found


In [4]:
# Setting status=None allows for the old version to be retrieved
try:
    potential = am.load_lammps_potential(id='2004--Zhou-X-W--Ni--LAMMPS--ipr1', status=None)
except Exception as ex:
    print(f'{type(ex).__name__}: {ex.args[0]}')
else:
    print(potential)

potential_LAMMPS record named 2004--Zhou-X-W--Ni--LAMMPS--ipr1


#### 2.1.2. By potid

You can also search using the potential model id - the main identifier for potentials on the Interatomic Potentials Reporitory - using the potid parameter.  If there is more than one "active" version of the potential, then a prompt list will appear. 

In [5]:
potential = am.load_lammps_potential(potid='2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Ni')
print(potential)

Multiple matching records retrieved from local
1 2004--Zhou-X-W--Ni--LAMMPS--ipr2 Ni
2 EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulation_Ni__MO_593762436933_000 Ni
3 EAM_Dynamo_ZhouJohnsonWadley_2004_Ni__MO_110256178378_005 Ni


Please select one: 1


potential_LAMMPS record named 2004--Zhou-X-W--Ni--LAMMPS--ipr2


#### 2.1.3. By category

The LAMMPS potentials can also be (partially) explored by using the pair_style, elements, and symbols parameters.  For elements vs. symbols

- elements are the specific elements that potentials for modeling atoms are representing. A model may not be associated with an element if it is a meta or coarse-grain model.
- symbols are the model names for each unique interaction. For atomic models, this is usually the same as elements but can differ if multiple models for the same element are part of the same potential.   

In [6]:
###### Select an eam-style potential for Ni
potential = am.load_lammps_potential(pair_style=['eam', 'eam/alloy', 'eam/fs'], elements='Ni')
print(potential)

Multiple matching records retrieved from local
1 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1 Cu-Ni
2 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1 Ag-Au-Cu-Ni-Pd-Pt
3 1986--Foiles-S-M--Ni--LAMMPS--ipr1 Ni
4 1987--Ackland-G-J--Ni--LAMMPS--ipr1 Ni
5 1987--Ackland-G-J--Ni--LAMMPS--ipr2 Ni
6 1989--Adams-J-B--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1 Ag-Au-Cu-Ni-Pd-Pt
7 1989--Adams-J-B--Ni--LAMMPS--ipr1 Ni
8 1995--Angelo-J-E--Ni-Al-H--LAMMPS--ipr1 Ni-Al-H
9 1999--Mishin-Y--Ni--LAMMPS--ipr1 Ni
10 2002--Mishin-Y--Ni-Al--LAMMPS--ipr1 Ni-Al
11 2004--Mishin-Y--Ni-Al--LAMMPS--ipr1 Ni-Al
12 2004--Mishin-Y--Ni-Al--LAMMPS--ipr2 Ni-Al
13 2004--Zhou-X-W--Ni--LAMMPS--ipr2 Ni
14 2009--Bonny-G--Fe-Cu-Ni--LAMMPS--ipr1 Fe-Cu-Ni
15 2009--Bonny-G--Fe-Ni--LAMMPS--ipr1 Fe-Ni
16 2009--Purja-Pun-G-P--Ni-Al--LAMMPS--ipr1 Ni-Al
17 2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr1 Fe-Ni-Cr
18 2011--Bonny-G--Fe-Ni-Cr--LAMMPS--ipr2 Fe-Ni-Cr
19 2012--Mendelev-M-I--Ni--LAMMPS--ipr1 Ni
20 2012--Mendelev-M-I--Ni-Zr--LAMMPS--ipr1 Ni-Zr
21 2013--

Please select one: 16


potential_LAMMPS record named 2009--Purja-Pun-G-P--Ni-Al--LAMMPS--ipr1


### 2.2. Using Database class

*Added version 1.3.3*

*Updated version 1.4.0* for potentials package version 0.3.0 changes.

More involved searches can be performed using the atomman.library.Database class.  This class inherits from [potentials.Database](https://github.com/lmhale99/potentials/blob/master/doc/4.%20Database%20Class.ipynb) and has all of the methods for exploring the potentials and LAMMPS potentials entries as the parent class.

In [7]:
potdb = am.library.Database()
lammps_potentials, lammps_potentials_df = potdb.get_lammps_potentials(return_df=True, verbose=True)
print('The first loaded LAMMPS potential is', lammps_potentials[0])

Found 517 matching potential_LAMMPS records in local library
Found 450 matching potential_LAMMPS_KIM records in local library
Built 561 lammps potentials for KIM models
The first loaded LAMMPS potential is potential_LAMMPS record named 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1


In [8]:
lammps_potentials_df

,name,id,key,url,potid,potkey,poturl,units,atom_style,allsymbols,pair_style,status,symbols,elements,artifacts,comments,dois
0,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr1,062d2ba7-3903-40ae-a772-daa471d107c6,062d2ba7-3903-40ae-a772-daa471d107c6,1985--Foiles-S-M--Ni-Cu,301f04ce-9082-4542-8590-489300cd19e8,NaN,metal,atomic,0,eam,active,"[Cu, Ni]","[Cu, Ni]","[{'filename': 'Cu_smf7.eam', 'label': None, 'u...",Potential 1985--Foiles-S-M--Ni-Cu--LAMMPS--ipr...,[10.1103/physrevb.32.7685]
1,1985--Stillinger-F-H--Si--LAMMPS--ipr1,1985--Stillinger-F-H--Si--LAMMPS--ipr1,d085648c-b3ef-4be8-824b-7093fd22770a,d085648c-b3ef-4be8-824b-7093fd22770a,1985--Stillinger-F-H-Weber-T-A--Si,edc31ad6-2b9a-455c-9b5f-e888a672ecbd,NaN,metal,atomic,0,sw,active,[Si],[Si],"[{'filename': 'Si.sw', 'label': None, 'url': '...",Potential 1985--Stillinger-F-H--Si--LAMMPS--ip...,"[10.1103/physrevb.31.5262, 10.1103/physrevb.33..."
2,1986--Foiles-S-M--Ag--LAMMPS--ipr1,1986--Foiles-S-M--Ag--LAMMPS--ipr1,76a265fc-45ff-49d7-8c64-2044f12402f2,76a265fc-45ff-49d7-8c64-2044f12402f2,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag,672d54f8-9f48-4200-af56-8a7378ebbc4a,NaN,metal,atomic,0,eam,active,[Ag],[Ag],"[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
3,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt--LAMMPS--ipr1,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,c5afa7e8-6b3b-49cd-ad1c-ae3e4329363a,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Ag-Au-Cu-...,7a1302de-59cf-4efb-900e-cad845b68ee5,NaN,metal,atomic,0,eam,active,"[Ag, Au, Cu, Ni, Pd, Pt]","[Ag, Au, Cu, Ni, Pd, Pt]","[{'filename': 'Ag_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Ag-Au-Cu-Ni-Pd-Pt-...,[10.1103/physrevb.33.7983]
4,1986--Foiles-S-M--Au--LAMMPS--ipr1,1986--Foiles-S-M--Au--LAMMPS--ipr1,c588810a-b96d-4871-bfe2-cff8a5a7c709,c588810a-b96d-4871-bfe2-cff8a5a7c709,1986--Foiles-S-M-Baskes-M-I-Daw-M-S--Au,ffb66faa-319d-4556-8363-dad3959cd553,NaN,metal,atomic,0,eam,active,[Au],[Au],"[{'filename': 'Au_u3.eam', 'label': None, 'url...",Potential 1986--Foiles-S-M--Au--LAMMPS--ipr1 l...,[10.1103/physrevb.33.7983]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1073,MO_987541074959,EMT_Asap_MetalGlass_PaduraruKenoufiBailey_2007...,MO_987541074959_001,https://potentials.nist.gov/pid/rest/local/pot...,2007--Paduraru-A-Kenoufi-A-Bailey-N-P-Schiotz-...,d05c1f39-89ef-47c7-b263-7bf0145b36ac,https://potentials.nist.gov/pid/rest/local/pot...,metal,atomic,0,kim,active,"[Cu, Zr]","[Cu, Zr]",NaN,NaN,NaN
1074,MO_988703794028,EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulat...,MO_988703794028_000,https://potentials.nist.gov/pid/rest/local/pot...,2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Pb,1c592eb1-3cdd-4090-aa0b-e7246f7d5f91,https://potentials.nist.gov/pid/rest/local/pot...,metal,atomic,0,kim,active,[Pb],[Pb],NaN,NaN,NaN
1075,MO_993644691224,EAM_Dynamo_ZhouJohnsonWadley_2004NISTretabulat...,MO_993644691224_000,https://potentials.nist.gov/pid/rest/local/pot...,2004--Zhou-X-W-Johnson-R-A-Wadley-H-N-G--Pd,5ea5dd6c-a901-4acd-b036-9c33a0acd5b0,https://potentials.nist.gov/pid/rest/local/pot...,metal,atomic,0,kim,active,[Pd],[Pd],NaN,NaN,NaN
1076,MO_995724792024,LJ_Shifted_Bernardes_1958LowCutoff_Kr__MO_9957...,MO_995724792024_004,https://potentials.nist.gov/pid/rest/local/pot...,1958--Bernardes-N--Kr,e62cbc12-99ba-469b-8d4e-e26ad40cb15a,https://potentials.nist.gov/pid/rest/local/pot...,metal,atomic,0,kim,active,[Kr],[Kr],NaN,NaN,NaN


### 2.3. From data model

*Updated version 1.4.0* - atomman.lammps.Potential() is now a wrapper that returns either a potentials.record.PotentialLAMMPS or a potentials.record.PotentialLAMMPSKIM object.

Potentials can also be directly loaded from a potential_LAMMPS data model record using atomman.lammps.Potential().  The data models can exist as Python dictionaries or be represented in JSON/XML formatted strings or files.  

Parameters

- __model__ (*DataModelDict, str, or file-like object*) A JSON/XML data model containing a potential-LAMMPS or a potential-LAMMPS-KIM branch.
- __name__ (*str, optional*) The record name to use.  If not given, this will be set to the potential's id.
- __pot_dir__ (*str, optional*) The path to a directory containing any artifacts associated with the potential.  Default value is None, which assumes any required files will be in the working directory when LAMMPS is executed.
- __kim_id__ (*str, optional*) The full KIM model id indicating the version of a KIM model to use. If not given, then the newest known version will be used.


This is a demonstration data model record for an eam/alloy style potential.  NOT A REAL RECORD!

- See [Section 4](#section4) for tools to build the data models for most common LAMMPS pair_styles
- See [Section 5](#section5) for specific details on the data model record schema and examples.

In [16]:
# Define data model record in JSON format
eam_alloy_json = """{
    "potential-LAMMPS": {
        "key": "a45a7731-d115-4079-b6f5-aa700c5b5c56",
        "id": "EAM-demo--LAMMPS--v1",
        "potential": {
            "key": "820738a9-f556-468b-9041-9d98351ff751",
            "id": "EAM-demo"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "symbol": "Ni",
                "element": "Ni",
                "mass": 58.6934
            },
            {
                "symbol": "Al",
                "element": "Al",
                "mass": 26.981539
            },
            {
                "symbol": "Co",
                "element": "Co",
                "mass": 58.9332
            }
        ],
        "pair_style": {
            "type": "eam/alloy"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.eam.alloy"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}"""

Load the potential from the record.

In [17]:
potential = lmp.Potential(eam_alloy_json)
print(potential)

potential_LAMMPS record named EAM-demo--LAMMPS--v1


## 3. Potential Classes <a id='section3'></a>

*Updated version 1.4.0*: atomman.lammps.Potential() now returns a potentials.record.PotentialLAMMPS or a potentials.record.PotentialLAMMPSKIM object.

See the potentials package documentation for more details on the [base Record class](https://github.com/lmhale99/potentials/blob/master/doc/5.%20Record%20Classes.ipynb), the [PotentialLAMMPS subclass](https://github.com/lmhale99/potentials/blob/master/doc/5.3.%20LAMMPS%20potentials.ipynb), and the [PotentialLAMMPSKIM subclass](https://github.com/lmhale99/potentials/blob/master/doc/5.4.%20OpenKIM%20models.ipynb).

The objects for the loaded potentials have a number of attributes and methods that

- Uniquely identify the LAMMPS potential version and the conceptual potential model that it is based on.
- Specify the LAMMPS settings to use with the potential.
- Define metadata for all unique atom models, i.e. atom type properties.
- Dynamically construct LAMMPS command lines for the potential based on the list of model symbols found in an atomic system.

### 3.1. Potential identifiers and database metadata

Each LAMMPS potential is identified according to a potential model, i.e. the concept of the potential, and the implementation, i.e. the version of the parameter file(s). The potential is further characterized with a status that allows for older, less preferred and erroneous implementations to be filtered out.

Here are some of the basic identifiers

- __potkey__ (*str*) uuid4 hash-key for the potential model, i.e. the published mathematical form.  This is permanent once assigned.
- __potid__ (*str*) human-readable unique identifier for the potential model derived from year, author, element model information.  This ideally is permanent, but may change based on updated publication information.
- __key__ (*str*) uuid hash-key for the LAMMPS potential implementation, i.e. the specific potential parameter file(s) and/or LAMMPS pair_style/pair_coeff command lines used.  This is permament once assigned.
- __id__ (*str*) human-readable identifier for the LAMMPS potential implementation derived from the potid, source and version.  This may change if potid changes.
- __status__ (*str*) indicates the current status of the potential implementation
    - 'active' = any implementation that is not known to be incorrect or superceded by a newer version. 
    - 'superceded' = a correctly implemented version that has been replaced by a newer version to address minor issues. 
    - 'retracted' = an incorrectly implemented version due to either critical errors with the parameterization, file format, or metadata in the associated potential_LAMMPS record. 

In [18]:
print("potential.potkey ->", potential.potkey)
print("potential.potid -> ", potential.potid)
print("potential.key ->   ", potential.key)
print("potential.id ->    ", potential.id)
print("potential.status ->", potential.status)

potential.potkey -> 820738a9-f556-468b-9041-9d98351ff751
potential.potid ->  EAM-demo
potential.key ->    a45a7731-d115-4079-b6f5-aa700c5b5c56
potential.id ->     EAM-demo--LAMMPS--v1
potential.status -> active


The string representation of the Potential returns its id.

In [19]:
print(potential)

potential_LAMMPS record named EAM-demo--LAMMPS--v1


### 3.2. LAMMPS settings

A number of attributes are used to specify required settings to use with the potentials.

- __atom_style__ (*str*) LAMMPS atom_style option for the potential.
- __pair_style__ (*str*) LAMMPS pair_style option for the potential.
- __units__ (*str*) LAMMPS units option for the potential.
- __allsymbols__ (*bool*) flag that indicates if the pair_style requires that pair_coeff be specified for all interaction models even if some models are not actually use

In [20]:
print("potential.pair_style ->", potential.pair_style)
print("potential.atom_style ->", potential.atom_style)
print("potential.units ->     ", potential.units)
print("potential.allsymbols ->", potential.allsymbols)

potential.pair_style -> eam/alloy
potential.atom_style -> atomic
potential.units ->      metal
potential.allsymbols -> False


### 3.3. Atom-type properties

Each unique atomic model is associated with a symbol, and that symbol can then be associated with an element, a mass and/or a charge.

- __symbols__ (*list*) The symbols used to uniquely identify the different atomic models defined by the potential.
- __elements()__ (*list*) Returns the list of elements that correspond to the symbols. If a symbol is not associated with an element, then that symbol's element is given as None.
- __masses()__ (*list*) The atomic/particle masses that correspond to the symbols.  For symbols associated with elements, the class will return the standard elemental mass for that element if the mass is not explicitly set elsewhere.
- __charges()__ (*list*) The atomic/particle charges that correspond to the symbols.  This is used for potentials that assign constant charges to particles based on their symbols, e.g. Coulumbic potentials.  Charges for potentials with variable charges, e.g. ReaxFF and COMB, should not be set. If no charge is set for a symbol, the charge is assumed to be 0.

The elements(), masses(), and charges() methods all take the same parameters

- __symbols__ (*str or list, optional*) a list of the potential's symbol models to retrieve values for. If value is None (default) then all values are returned according to the symbols attribute of the class.

Calling the methods without parameters uses all symbols.

In [21]:
# Calling methods without parameters uses all symbols
print("potential.symbols ->   ", potential.symbols)
print("potential.elements() ->", potential.elements())
print("potential.charges() ->  ", potential.charges())
print("potential.masses() ->  ", potential.masses())

potential.symbols ->    ['Ni', 'Al', 'Co']
potential.elements() -> ['Ni', 'Al', 'Co']
potential.charges() ->   [0.0, 0.0, 0.0]
potential.masses() ->   [58.6934, 26.981539, 58.9332]


Calling the methods with symbols gives values for that particular set.

In [22]:
# Calling methods with symbols uses that particular set
print("potential.elements('Ni') ->", potential.elements('Ni'))
print("potential.charges('Ni') ->", potential.charges('Ni'))
print("potential.masses('Ni') ->", potential.masses('Ni'))
print()

print("potential.elements(['Al', 'Al', 'Co', 'Al']) ->", potential.elements(['Al', 'Al', 'Co', 'Al']))
print("potential.charges(['Al', 'Al', 'Co', 'Al']) ->", potential.charges(['Al', 'Al', 'Co', 'Al']))
print("potential.masses(['Al', 'Al', 'Co', 'Al']) ->  ", potential.masses(['Al', 'Al', 'Co', 'Al']))
print()

potential.elements('Ni') -> ['Ni']
potential.charges('Ni') -> [0.0]
potential.masses('Ni') -> [58.6934]

potential.elements(['Al', 'Al', 'Co', 'Al']) -> ['Al', 'Al', 'Co', 'Al']
potential.charges(['Al', 'Al', 'Co', 'Al']) -> [0.0, 0.0, 0.0, 0.0]
potential.masses(['Al', 'Al', 'Co', 'Al']) ->   [26.981539, 26.981539, 58.9332, 26.981539]



### 3.4. Generate LAMMPS commands

#### 3.4.1. Potential directory

For potentials with parameter files, LAMMPS needs to know where the files are in order to read them in. With the Potential class, this is handled with the pot_dir parameter

- **pot_dir** (*str or None*) the directory containing files associated with the potential.  If None, then the files are assumed to either be in the working directory or a directory that LAMMPS knows about.


In [23]:
print("potential.pot_dir ->  ", potential.pot_dir)

potential.pot_dir ->   


Depending on how the potential was loaded, the default value of pot_dir may be different:

- If load_lammps_potential() or Database.get_lammps_potential(s) are used, the pot_dir value will depend on the pot_dir_setting.
- If the potential is loaded directly from a file, then the default pot_dir value is ''.

__NOTE__: The pot_dir attribute can be directly set to a different value at any time after the object has been created.

In [24]:
potential.pot_dir = '/demo/new_potential_folder'

print("potential.pot_dir -> ", potential.pot_dir)

potential.pot_dir ->  /demo/new_potential_folder


#### 3.4.2. pair_info()

The LAMMPS input command lines for using the potential can be autogenerated using the pair_info() method.

Parameters

- __symbols__ (*str or list, optional*) a list of the potential's symbol models to associate with integer atom types of a system. If value is None (default) then Potential.symbols is used.
- __masses__ (*list, optional*) Can be given to override the default symbol-based masses for each atom type.  Must be a list of the same length as symbols.  Any values of None in the list indicate that the default value be used for that atom type.  *Added version 1.3.0*.
- __prompt__ (*bool, optional*) If True (default), then a screen prompt will appear asking for the isotope number if no mass is pre-defined for a symbol and the associated element lacks a single standard atomic/ionic mass.  If False, then an error will be raised for these cases instead. *Added version 1.4.0*.
- __comments__ (*bool, optional*) Indicates if print command lines detailing information on the potential are to be included.  Default value is True. *Added version 1.4.0*.
 
**NOTE:** For pair_styles hybrid and hybrid/overlay, the list of symbols given as parameters must include all defined atom model interactions.

Without symbols gives LAMMPS commands for all symbols in the same order as the data model.

In [25]:
print('potential.pair_info() ->')
print(potential.pair_info())

potential.pair_info() ->
pair_style eam/alloy
pair_coeff * * /demo/new_potential_folder/file.eam.alloy Ni Al Co
mass 1 58.6934
mass 2 26.981539
mass 3 58.9332




Give a list of symbols to dynamically generate commands based on the elements in the associated atomic system.

In [26]:
print("potential.pair_info('Ni') ->")
print(potential.pair_info('Ni'))

potential.pair_info('Ni') ->
pair_style eam/alloy
pair_coeff * * /demo/new_potential_folder/file.eam.alloy Ni
mass 1 58.6934




In [27]:
print("potential.pair_info(['Al', 'Al', 'Co', 'Al']) ->")
print(potential.pair_info(['Al', 'Al', 'Co', 'Al']))

potential.pair_info(['Al', 'Al', 'Co', 'Al']) ->
pair_style eam/alloy
pair_coeff * * /demo/new_potential_folder/file.eam.alloy Al Al Co Al
mass 1 26.981539
mass 2 26.981539
mass 3 58.9332
mass 4 26.981539




Specify masses to override default values

In [28]:
print("potential.pair_info('Ni', masses=27) ->")
print(potential.pair_info('Ni', masses=27))

potential.pair_info('Ni', masses=27) ->
pair_style eam/alloy
pair_coeff * * /demo/new_potential_folder/file.eam.alloy Ni
mass 1 27




#### 3.4.3. System.dump('atom_data')

The pair info command lines can also be directly incorporated with the read atom info command lines generated when a System is dumped to a LAMMPS data file by giving the potential to the dump function.  This is highly preferred for writing calculation methods to be agnostic to the potential as it ensures that the data file is compatible with the potential and the command lines are in the proper order. 

In [29]:
# Define a simple unit cell
fcc_test = am.load('prototype', 'A1--Cu--fcc', a=3.52, symbols='Ni')

# Dump to data file
atom_pair_info = fcc_test.dump('atom_data', f='test.dat', potential=potential)

# Check the command lines
print(atom_pair_info)

# Delete dump file
Path('test.dat').unlink()

units metal
atom_style atomic

boundary p p p
read_data test.dat

pair_style eam/alloy
pair_coeff * * /demo/new_potential_folder/file.eam.alloy Ni
mass 1 58.6934




## 4. Building a new potential<a id='section4'></a>

*Added version 1.3.4*

The function build_lammps_potential() is also inherited from potentials allowing for users to easily build and save their own potential objects.    

See the documentation for the [potentials package](https://github.com/usnistgov/potentials) for more details and examples.

Parameters for all formats

- __pair_style__ (*str*) The LAMMPS pair_style option to use.
- __id__ (*str, optional*) A human-readable identifier to name the LAMMPS potential implementation.  Must be set in order to save to the database as the id is used as the potential's file name.
- __key__ (*str, optional*) A UUID4 code to uniquely identify the LAMMPS potential implementation.  If not specified, a new UUID4 code is automatically generated.
- __potid__ (*str, optional*) A human-readable identifier to refer to the conceptual potential model that the potential is based on.  This should be shared by alternate implementations of the same potential.
- __potkey__ (*str, optional*) A UUID4 code to uniquely identify the conceptual potential model. This should be shared by alternate implementations of the same potential. If not specified, a new UUID4 code is automatically generated.
- __units__ (*str, optional*) The LAMMPS units option to use.
- __atom_style__ (*str, optional*) The LAMMPS atom_style option to use.
- __pair_style_terms__ (*list, optional*) Any other terms that appear on the pair_style line (like cutoff) if needed.
- __status__ (*str, optional*) Indicates if the implementation is 'active' (valid and current), 'superseded' (valid, but better ones exist), or 'retracted' (invalid). Default value is 'active'.
- __allsymbols__ (*bool, optional*) Flag indicating if the coefficient lines must be defined for every particle model in the potential even if those particles are not used.  Default value is False as most pair_styles do not require this.
- __elements__ (*str or list, optional*) The elemental symbols associated with each particle model if the particles represent atoms.
- __masses__ (*float or list, optional*) The masses of each particle.  Optional if elements is given as standard values can be used.
- __charges__ (*float or list, optional*) The static charges to assign to each particle, if the model calls for it.
- __symbols__ (*str or list, optional*) The symbols used to identify each unique particle model. Optional if elements is given and the particle symbols are the same as the elemental symbols.
- __command_terms__ (*list, optional*) Allows any other LAMMPS command lines that must be set for the potential to work properly to be set.  Each command line should be given as a list of terms, and multiple command lines given as a list of lists.

Parameters for true pair potentials, e.g. lj/cut

- __interactions__ (*dict or list of dict, optional*) Each unique pair interaction is characterized by a dict containing symbols=two element model symbols, and terms=list of pair_coeff terms.

Parameters for potentials with a single parameter file, e.g. eam/alloy

- __paramfile__ (*str, optional*) The name of the potential's parameter file.
            
Parameters for potentials with common library and specific parameter files, e.g. meam and eim

- __libfile__ (*str, optional*) The name of the potential's library file.
- __paramfile__ (*str, optional*) The name of the potential's parameter file.
            
Parameters for the original eam style

- __paramfiles__ (*str or list, optional*) The name(s) of the potential's parameter file(s).  There should be one parameter file for each element model.


__NOTE__: This function currently supports *most but not all* LAMMPS pair_styles. If the function says that the pair_style is unsupported but it has a format similar to one of the listed options, you can call the associated builder class in potentials directly (and let us know it needs to be added to the lists).  If the format is decidedly different (e.g. hybrid), then the underlying information can be manually built as detailed in Section #6 below.  

Initialize a builder for an eam/alloy potential. Note that only the pair style, list of elements and parameter file(s) are needed to use it.

In [33]:
custom_eam_potential = am.build_lammps_potential(pair_style='eam/alloy', 
                                                 elements='Ag', symbols='Ag',
                                                 paramfile='Ag-custom.eam.alloy')

Generate a potential object from the builder

In [34]:
potential = custom_eam_potential.potential()

print(potential.pair_info())

pair_style eam/alloy
pair_coeff * * Ag-custom.eam.alloy Ag
mass 1 107.8682




If an id is also assigned to the potential, then it can be saved to the local database with Database.save_lammps_potentials().

In [36]:
custom_eam_potential = am.build_lammps_potential(pair_style='eam/alloy', 
                                                 id='custom-eam--LAMMPS--v1',
                                                 potid='custom-eam',
                                                 elements='Ag',
                                                 symbols='Ag',
                                                 paramfile='Ag-custom.eam.alloy')

## 5. potential_LAMMPS Data Model <a id='section5'></a>

This section describes the structure of the potential_LAMMPS data model and provides some examples for different LAMMPS pair styles.

### 5.1. potential_LAMMPS data model schema <a id='section5'></a>

The potential_LAMMPS data model is consistent with a json/xml structure. The root element of the model is __"potential-LAMMPS"__.

Elements: 

- __"key"__ : a uuid hash-key that uniquely identifies the potential implementation.  Corresponds to the Potential.key attribute.
- __"id"__ : a human-readable identifier that uniquely identifies the potential implementation.  Corresponds to the Potential.id attribute.
- __"potential"__ : contains metadata associated with the potential model.
- __"units"__ : the LAMMPS units option to use with the potential.  Corresponds to the Potential.units attribute.
- __"atom_style"__ : the LAMMPS atom_style option to use with the potential.  Corresponds to the Potential.atom_style attribute.
- __"allsymbols"__ : boolean flag that indicates if all of a potential's symbols need to be listed in the pair_coeff lines.  This is required for certain pair_styles.  Assumed to be False if field is missing.
- __"status"__ : label characterizing the known status of the potential, i.e. "active", "superceded", or "retracted".  This makes it possible to separate old/bad implementations from current ones.  If field is missing, status is assumed to be "active".
- __"atom"__ : provides information relating to the atom-models defined in the potential.
- __"pair_style"__ : classifies the terms that appear in the LAMMPS pair_style command associated with the potential.
- __"pair_coeff"__ : classifies the terms that appear in the LAMMPS pair_coeff command associated with the potential.
- __"command"__ : classifies the terms that appear in any other LAMMPS command associated with the potential.

#### 5.1.1. The "potential" branch <a id='section5.1.1'></a>

The __"potential"__ branch gives metadata associated with the potential model that the LAMMPS implementation is based on. The potential model is the concept (mathematical expression) of a potential, and multiple implementations of the same potential model may exist. 

Subelements:

- __"key"__ : a uuid hash-key that uniquely identifies the potential model.  Corresponds to the Potential.potkey attribute.
- __"id"__ : a human-readable identifier that uniquely identifies the potential model.   Corresponds to the Potential.potid attribute.

In [37]:
print("""{
    "potential": {
        "key": <uuid>, 
        "id": <name>
    }
}""")

{
    "potential": {
        "key": <uuid>, 
        "id": <name>
    }
}


#### 5.1.2. The "atom" branch <a id='section5.1.2'></a>

The __"atom"__ branch of the data model provides a list of the atom-models described by the potential.

Subelements:

- __"element"__: chemical element tag associated with the atom-model. If not given, it is set equal to the __"symbol"__ value.
- __"symbol"__: unique symbol associated with the atom-model. If not given, it is set equal to the __"element"__ value.
- __"mass"__: element mass associated with the element/atom-model. If not given, it is set equal to the standard atomic mass value associated with the element given by __"element"__. 
- __"charge"__: ionic charge associated with the atom-model. If not given, it is set to 0.0.  Only needs to be defined for pair_styles where the ionic charge is constant for a given atom-model e.g. Columbic potentials.

In [38]:
# Single-element potentials
print("""{
    "atom": {
        "element": <element-tag>,
        "symbol": <atom-model-name>,
        "mass": <element-mass>,
        "charge": <ionic-charge>
    }
}""")

{
    "atom": {
        "element": <element-tag>,
        "symbol": <atom-model-name>,
        "mass": <element-mass>,
        "charge": <ionic-charge>
    }
}


In [39]:
# Multi-element potentials
print("""{
    "atom": [
        {
            "element": <element-tag>,
            "symbol": <atom-model-name>,
            "mass": <element-mass>,
            "charge": <ionic-charge>
        },
        {
            "element": <element-tag>,
            "symbol": <atom-model-name>,
            "mass": <element-mass>,
            "charge": <ionic-charge>
        }
    ]
}""")

{
    "atom": [
        {
            "element": <element-tag>,
            "symbol": <atom-model-name>,
            "mass": <element-mass>,
            "charge": <ionic-charge>
        },
        {
            "element": <element-tag>,
            "symbol": <atom-model-name>,
            "mass": <element-mass>,
            "charge": <ionic-charge>
        }
    ]
}


#### 5.1.3. The "pair_style" branch <a id='section5.1.3'></a>

The __"pair_style"__ branch of the data model characterizes the terms that appear in the LAMMPS pair_style command for the potential.

Subelements:

- __"type"__ : the specific LAMMPS pair_style option for the potential.  Corresponds to the Potential.pair_style attribute.
- __"term"__ : a list characterizing any additional terms that appear in the pair_style command line. The available subelements are the same as the ones available in the __"term"__ branch of __"pair_coeff"__.
    

In [40]:
print("""{
    "pair_style": {
        "type": <pair_style-type>,
        "term": [
            {
                "parameter": <number>
            },
            {
                "option": <string>
            }
        ]
    } 
}""")

{
    "pair_style": {
        "type": <pair_style-type>,
        "term": [
            {
                "parameter": <number>
            },
            {
                "option": <string>
            }
        ]
    } 
}


#### 5.1.4. The "pair_coeff" branch <a id='section5.1.4'></a>

The __"pair_coeff"__ branch of the data model characterizes the terms that appear in the LAMMPS pair_coeff command lines for the potential. Multiple __"pair_coeff"__ values can be given.

Subelements:

- __"interaction"__ : an __"interaction"__ branch that outlines the atom-model interactions associated with a pair_coeff command line. 
    - __"symbol"__ : a list of the atom-model symbols that the interaction is associated with.
- __"term"__ : a __"term"__ branch that characterizes all the terms that appear in a pair_coeff command line.
    - __"option"__ : a string option value.
    - __"parameter"__ : a numerical parameter value.
    - __"file"__ : a file name. Used by potential styles that read parameters from external files, like eam/alloy. 
    - __"symbols"__ : Boolean indicating to show a list of the atom-model symbols to use. Used by potential styles that read parameters from external potential files, like eam/alloy. 
    - __"symbolsList"__ : Boolean indicating to show a list of the unique atom-model symbols to use.  Don't use!
    
Different types of pair_styles have slightly different rules with interpreting the __"pair_coeff"__ branch.

- For two-body style potentials, every unique atom-model pair has its own __"pair_coeff"__ element. __"interaction"__-__"symbol"__ must have two values indicating the atom-model pair, eg. ["Ag", "Ag"] for the Ag-Ag interaction and ["Ag", "Cu"] for Ag-Cu and Cu-Ag interactions. __"interaction"__ is optional if there is only one pair interaction, i.e. an elemental pair potential.

- For many-body style potentials, __"interaction"__ is optional as all atom-model interactions are accessed using the same pair_coeff command line.

- For hybrid style potentials, __"interaction"__-__"symbol"__ lists all atom-model symbols associated with the particular sub-style. It will have two values for pair sub-styles, but can be of any length for many-body sub-styles.

In [41]:
# pair_coeff example for two-body style potentials 
print("""{
    "pair_coeff": [
        {
            "interaction": {
                "symbol": [
                    <atom-model-name>,
                    <atom-model-name>
                ]
            }
            "term": [
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "option": <string>
                }
            ]
        },
        {
            "interaction": {
                "symbol": [
                    <atom-model-name>,
                    <atom-model-name>
                ]
            }
            "term": [
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "option": <string>
                }
            ]
        }        
    } 
}""")

{
    "pair_coeff": [
        {
            "interaction": {
                "symbol": [
                    <atom-model-name>,
                    <atom-model-name>
                ]
            }
            "term": [
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "option": <string>
                }
            ]
        },
        {
            "interaction": {
                "symbol": [
                    <atom-model-name>,
                    <atom-model-name>
                ]
            }
            "term": [
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
                {
                    "parameter": <number>
                },
        

In [42]:
# pair_coeff example for many-body potentials with library and parameter files
print("""{
    "pair_coeff": {
        "term": [
            {
                "file": <file-path>
            },
            {
                "option": <string>
            },
            {
                "file": <file-path>
            },
            {
                "symbols": true
            }
        ]
    } 
}""")

{
    "pair_coeff": {
        "term": [
            {
                "file": <file-path>
            },
            {
                "option": <string>
            },
            {
                "file": <file-path>
            },
            {
                "symbols": true
            }
        ]
    } 
}


#### 5.1.5. The "command" branch <a id='section5.1.5'></a>

The __"command"__ branch of the data model characterizes the terms that appear in any other LAMMPS command lines that are required for properly running the potential. This is needed for specifying such things as a fix qeq/comb for COMB potentials.

Subelements:

- __"term"__ : a __"term"__ branch that characterizes all the terms that appear in the command line.
    - __"option"__ : a string option value.
    - __"parameter"__ : a numerical parameter value.
    - __"file"__ : a file name.
    - __"symbols"__ : Boolean indicating to show a list of the atom-model symbols to use. 
    - __"symbolsList"__ : Boolean indicating to show a list of the unique atom-model symbols to use. 

In [43]:
print("""{
    "command": {
        "term": [
            {
                "option": <string>
            },
            {
                "option": <string>
            },
            {
                "parameter": <number>
            },
            {
                "parameter": <number>
            }                
        ]
    }
}""")

{
    "command": {
        "term": [
            {
                "option": <string>
            },
            {
                "option": <string>
            },
            {
                "parameter": <number>
            },
            {
                "parameter": <number>
            }                
        ]
    }
}


### 5.2. Example data models <a id='section5.2'></a>

__WARNING!__ These are not real potentials! They are only meant to provide a demonstration for different potential styles.

#### 5.2.1. Lennard-Jones and other two-body style potentials <a id='section5.2.1'></a>

In [44]:
print("""{
    "potential-LAMMPS": {
        "key": "7102f7ec-3612-4665-ad7e-60de508b5f37",
        "id": "lj_cut-demo--LAMMPS--v1",
        "potential": {
            "key": "ebf17ffa-a5e7-41c5-8e6d-8e00eb7f5068",
            "id": "lj_cut-demo"
        },
        "units": "lj",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "He"
            },
            {
                "element": "Ar"
            }
        ],
        "pair_style": {
            "type": "lj/cut",
            "term": {
                "parameter": 10.0
            }
        },
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "He",
                        "He"
                    ]
                },
                "term": [
                    {
                        "parameter": 1.0
                    },
                    {
                        "parameter": 1.0
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "Ar",
                        "Ar"
                    ]
                },
                "term": [
                    {
                        "parameter": 2.0
                    },
                    {
                        "parameter": 2.0
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "He",
                        "Ar"
                    ]
                },
                "term": [
                    {
                        "parameter": 1.0
                    },
                    {
                        "parameter": 2.0
                    }
                ]
            }            
        ]
    }
}""")

{
    "potential-LAMMPS": {
        "key": "7102f7ec-3612-4665-ad7e-60de508b5f37",
        "id": "lj_cut-demo--LAMMPS--v1",
        "potential": {
            "key": "ebf17ffa-a5e7-41c5-8e6d-8e00eb7f5068",
            "id": "lj_cut-demo"
        },
        "units": "lj",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "He"
            },
            {
                "element": "Ar"
            }
        ],
        "pair_style": {
            "type": "lj/cut",
            "term": {
                "parameter": 10.0
            }
        },
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "He",
                        "He"
                    ]
                },
                "term": [
                    {
                        "parameter": 1.0
                    },
                    {
                        "parameter": 1.0
                    }
           

#### 5.2.2. eam/alloy and other many-body potentials from single parameter files <a id='section5.2.2'></a>

**NOTE:** Data models for the original eam pair_style are different (see [Section 5.2.6.](#section5.2.6))!

In [45]:
print("""{
    "potential-LAMMPS": {
        "key": "a45a7731-d115-4079-b6f5-aa700c5b5c56",
        "id": "EAM-demo--LAMMPS--v1",
        "potential": {
            "key": "820738a9-f556-468b-9041-9d98351ff751",
            "id": "EAM-demo"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "Ni",
                "mass": 58.6934
            },
            {
                "element": "Al",
                "mass": 26.981539
            },
            {
                "element": "Co",
                "mass": 58.9332
            }
        ],
        "pair_style": {
            "type": "eam/alloy"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.eam.alloy"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")

{
    "potential-LAMMPS": {
        "key": "a45a7731-d115-4079-b6f5-aa700c5b5c56",
        "id": "EAM-demo--LAMMPS--v1",
        "potential": {
            "key": "820738a9-f556-468b-9041-9d98351ff751",
            "id": "EAM-demo"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "Ni",
                "mass": 58.6934
            },
            {
                "element": "Al",
                "mass": 26.981539
            },
            {
                "element": "Co",
                "mass": 58.9332
            }
        ],
        "pair_style": {
            "type": "eam/alloy"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.eam.alloy"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}


#### 5.2.3. COMB and other potentials requiring additional commands <a id='section5.2.3'></a>

In [46]:
print("""{
    "potential-LAMMPS": {
        "key": "c6623d40-a3f2-4b2f-948a-3b277f174116",
        "id": "COMB-demo--LAMMPS--v2",
        "potential": {
            "key": "5ec2aac4-3dd4-4f5c-afb4-21749577c038",
            "id": "COMB-demo"
        },
        "units": "nano",
        "atom_style": "charge",
        "atom": [
            {
                "element": "O"
            },
            {
                "element": "Cu"
            },
            {
                "element": "Ni"
            }
        ],
        "pair_style": {
            "type": "comb",
            "term": {
                "option": "polar_off"
            }
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.comb"
                },
                {
                    "symbols": true
                }
            ]
        },
        "command": {
            "term": [
                {
                    "option": "fix"
                },
                {
                    "option": "qeq/comb"
                },
                {
                    "parameter": 10
                },
                {
                    "parameter": 0.0001
                }                
            ]
        }
    }
}""")

{
    "potential-LAMMPS": {
        "key": "c6623d40-a3f2-4b2f-948a-3b277f174116",
        "id": "COMB-demo--LAMMPS--v2",
        "potential": {
            "key": "5ec2aac4-3dd4-4f5c-afb4-21749577c038",
            "id": "COMB-demo"
        },
        "units": "nano",
        "atom_style": "charge",
        "atom": [
            {
                "element": "O"
            },
            {
                "element": "Cu"
            },
            {
                "element": "Ni"
            }
        ],
        "pair_style": {
            "type": "comb",
            "term": {
                "option": "polar_off"
            }
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "file.comb"
                },
                {
                    "symbols": true
                }
            ]
        },
        "command": {
            "term": [
                {
                    "option": "fix"
                },
               

#### 5.2.4. MEAM and other many-body potentials with library and parameter files <a id='section5.2.4'></a>

In [47]:
print("""{
    "potential-LAMMPS": {
        "key": "ac63aa71-808c-47e7-b80b-991a50870f35",
        "id": "MEAM-demo--LAMMPS--v1",
        "potential": {
            "key": "9546264a-06b8-451a-9920-f8a17cc6917b",
            "id": "MEAM-demo"
        },
        "units": "metal",
        "atom_style": "atom",
        "atom": [
            {
                "element": "Cu",
                "symbol": "CuD"
            },
            {
                "element": "Al",
                "symbol": "AlD"
            },
            {
                "element": "Fe",
                "symbol": "FeD"
            }
        ],
        "pair_style": {
            "type": "meam"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "library.meam"
                },
                {
                    "option": "CuD AlD FeD"
                },
                {
                    "file": "potential.meam"
                },
                {
                    "symbols": true
                }
            ]
        }
    }
}""")

{
    "potential-LAMMPS": {
        "key": "ac63aa71-808c-47e7-b80b-991a50870f35",
        "id": "MEAM-demo--LAMMPS--v1",
        "potential": {
            "key": "9546264a-06b8-451a-9920-f8a17cc6917b",
            "id": "MEAM-demo"
        },
        "units": "metal",
        "atom_style": "atom",
        "atom": [
            {
                "element": "Cu",
                "symbol": "CuD"
            },
            {
                "element": "Al",
                "symbol": "AlD"
            },
            {
                "element": "Fe",
                "symbol": "FeD"
            }
        ],
        "pair_style": {
            "type": "meam"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "library.meam"
                },
                {
                    "option": "CuD AlD FeD"
                },
                {
                    "file": "potential.meam"
                },
                {
                    

#### 5.2.5. hybrid-style potentials <a id='section5.2.5'></a>

In [48]:
print("""{
    "potential-LAMMPS": {
        "key": "7687807f-6355-4bef-bdc3-dc0dc944e106",
        "id": "hybrid-demo--LAMMPS--v3",
        "potential": {
            "key": "14226c15-561c-44d4-96ad-ad51304a3606",
            "id": "hybrid-demo"
        },
        "units": "metal",
        "atom_style": "atom",
        "atom": [
            {
                "element": "Cu"
            },
            {
                "element": "H"
            }
        ],
        "pair_style": {
            "type": "hybrid",
            "term": [
                {
                    "option": "eam/alloy"
                },
                {
                    "option": "lj/cut"
                },
                {
                    "parameter": 5.0
                }
            ]
        },
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "Cu"
                    ]
                },
                "term": [
                    {
                        "option": "eam/alloy"
                    },
                    {
                        "file": "cu.eam.alloy"
                    },
                    {
                        "symbols": true
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "Cu",
                        "H"
                    ]
                },
                "term": [
                    {
                        "option": "lj/cut"
                    },
                    {
                        "parameter": 3.5
                    },
                    {
                        "parameter": 3.0
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "H",
                        "H"
                    ]
                },
                "term": [
                    {
                        "option": "lj/cut"
                    },
                    {
                        "parameter": 1.2
                    },
                    {
                        "parameter": 2.4
                    }
                ]
            }
        ]
    }
}""")

{
    "potential-LAMMPS": {
        "key": "7687807f-6355-4bef-bdc3-dc0dc944e106",
        "id": "hybrid-demo--LAMMPS--v3",
        "potential": {
            "key": "14226c15-561c-44d4-96ad-ad51304a3606",
            "id": "hybrid-demo"
        },
        "units": "metal",
        "atom_style": "atom",
        "atom": [
            {
                "element": "Cu"
            },
            {
                "element": "H"
            }
        ],
        "pair_style": {
            "type": "hybrid",
            "term": [
                {
                    "option": "eam/alloy"
                },
                {
                    "option": "lj/cut"
                },
                {
                    "parameter": 5.0
                }
            ]
        },
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "Cu"
                    ]
                },
                "term": [
                  

#### 5.2.6. Original eam pair_style <a id='section5.2.6'></a>

In [49]:
print("""{
    "potential-LAMMPS": {
        "key": "3c437986-882f-4e23-bc39-6e12dcdba364", 
        "id": "eam-demo--LAMMPS--v2", 
        "potential": {
            "key": "5e92413c-8542-49c4-9608-b91ea5c12701", 
            "id": "eam-demo"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "element": "Cu", 
                "mass": 63.55
            }, 
            {
                "element": "Ni", 
                "mass": 58.71
            }
        ], 
        "pair_style": {
            "type": "eam"
        }, 
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "Cu", 
                        "Cu"
                    ]
                }, 
                "term": {
                    "file": "Cu.eam"
                }
            }, 
            {
                "interaction": {
                    "symbol": [
                        "Ni", 
                        "Ni"
                    ]
                }, 
                "term": {
                    "file": "Ni.eam"
                }
            }
        ]
    }
}""")

{
    "potential-LAMMPS": {
        "key": "3c437986-882f-4e23-bc39-6e12dcdba364", 
        "id": "eam-demo--LAMMPS--v2", 
        "potential": {
            "key": "5e92413c-8542-49c4-9608-b91ea5c12701", 
            "id": "eam-demo"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "element": "Cu", 
                "mass": 63.55
            }, 
            {
                "element": "Ni", 
                "mass": 58.71
            }
        ], 
        "pair_style": {
            "type": "eam"
        }, 
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "Cu", 
                        "Cu"
                    ]
                }, 
                "term": {
                    "file": "Cu.eam"
                }
            }, 
            {
                "interaction": {
                    "symbol": [
                        "Ni", 
   

### 5.2.7 Columbic pair_style

In [50]:
print("""{
    "potential-LAMMPS": {
        "key": "7f013ca0-0358-4721-ba72-790a074b0fd8",
        "id": "coul_long-demo--LAMMPS--v1",
        "potential": {
            "key": "3b326a65-5cbe-4a6c-862a-aa7203660b27",
            "id": "coul_long-demo"
        },
        "units": "metal",
        "atom_style": "charge",
        "atom": [
            {
                "element": "Na",
                "charge": 1
            },
            {
                "element": "Cl",
                "charge": -1
            }
        ],
        "pair_style": {
            "type": "coul/long",
            "term": {
                "parameter": 10.0
            }
        },
        "pair_coeff": {
            "term": []        
        }
    }
}""")

{
    "potential-LAMMPS": {
        "key": "7f013ca0-0358-4721-ba72-790a074b0fd8",
        "id": "coul_long-demo--LAMMPS--v1",
        "potential": {
            "key": "3b326a65-5cbe-4a6c-862a-aa7203660b27",
            "id": "coul_long-demo"
        },
        "units": "metal",
        "atom_style": "charge",
        "atom": [
            {
                "element": "Na",
                "charge": 1
            },
            {
                "element": "Cl",
                "charge": -1
            }
        ],
        "pair_style": {
            "type": "coul/long",
            "term": {
                "parameter": 10.0
            }
        },
        "pair_coeff": {
            "term": []        
        }
    }
}
